In [2]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

#lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['OGNUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        #data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        #resultado = md.backtesting(data, plot_flag = imprimo)
        resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


SPELLUSDT - Return [%]: 0.19% - # Trades: 2 - Profit Factor: nan - Win Rate [%]: 100.0
AXSUSDT - Return [%]: 0.69% - # Trades: 4 - Profit Factor: 4.801687685622774 - Win Rate [%]: 75.0


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:877: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)


ETHUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan

Error: unsupported operand type(s) for -: 'str' and 'str' - line: 21 - file: 1808796675.py

OGNUSDT - Return [%]: -8.12% - # Trades: 18 - Profit Factor: 0.06532170041963972 - Win Rate [%]: 27.77777777777778
CYBERUSDT - Return [%]: 0.1% - # Trades: 1 - Profit Factor: nan - Win Rate [%]: 100.0
1000PEPEUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
COMPUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
ADAUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
BCHUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan


RUNEUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
